# Basic usage of `pdpipe`

So how does using `pdpipe` looks like? Let's first import `pandas` and `pdpipe`, an intialize a nice little dataframe:


In [1]:
import pandas as pd
import pdpipe as pdp

In [2]:
df = pd.DataFrame(
    data=[
        [23, 'Jo', 'M', True, 0.07, 'USA', 'Living life to its fullest'],
        [23, 'Dana', 'F', True, 0.3, 'USA', 'the pen is mightier then the sword'],
        [25, 'Bo', 'M', False, 2.3, 'Greece', 'all for one and one for all'],
        [44, 'Derek', 'M', True, 1.1, 'Denmark', 'every life is precious'],
        [72, 'Regina', 'F', True, 7.1, 'Greece', 'all of you get off my porch'],
        [50, 'Jim', 'M', False, 0.2, 'Germany', 'boy do I love dogs and cats'],
        [80, 'Richy', 'M', False, 100.2, 'Finland', 'I gots the dollarz'],
        [80, 'Wealthus', 'F', False, 123.2, 'Finland', 'me likey them moniez'],
    ],
    columns=['Age', 'Name', 'Gender', 'Smoking', 'Savings', 'Country', 'Quote'],
)

This results in the following dataframe:

In [3]:
df

,Age,Name,Gender,Smoking,Savings,Country,Quote
0,23,Jo,M,True,0.07,USA,Living life to its fullest
1,23,Dana,F,True,0.30,USA,the pen is mightier then the sword
2,25,Bo,M,False,2.30,Greece,all for one and one for all
3,44,Derek,M,True,1.10,Denmark,every life is precious
4,72,Regina,F,True,7.10,Greece,all of you get off my porch
5,50,Jim,M,False,0.20,Germany,boy do I love dogs and cats
6,80,Richy,M,False,100.20,Finland,I gots the dollarz
7,80,Wealthus,F,False,123.20,Finland,me likey them moniez


## Constructing pipelines

We can create different pipeline stage object by calling their constructors,
which can be of course identified by their camel-cased names, such as 
`pdp.ColDrop` for dropping columns and `pdp.Encode` to encode them, etc.

To build a pipeline, we will usually call the `PdPipeline` class constructor,
and provide it with a list of pipeline stage objects:

In [4]:
pipeline = pdp.PdPipeline([
    pdp.ColDrop('Name'),
    pdp.drop_rows_where['Savings'] > 100,
    pdp.Bin({'Savings': [1]}, drop=False),
    pdp.Scale('StandardScaler'),
    pdp.TokenizeText('Quote'),
    pdp.SnowballStem('EnglishStemmer', columns=['Quote']),
    pdp.RemoveStopwords('English', 'Quote'),
    pdp.Encode('Gender'),
    pdp.OneHotEncode('Country'),
])

Printing the pipeline object displays it in order. 

In [5]:
pipeline

A pdpipe pipeline:
[ 0]  Drop columns Name
[ 1]  Drop rows by qualifier <RowQualifier: Qualify rows with df[Savings] >
      100>
[ 2]  Bin Savings by [1].
[ 3]  Scale columns Columns of dtypes <class 'numpy.number'>
[ 4]  Tokenize Quote
[ 5]  Stemming tokens in Quote...
[ 6]  Remove stopwords from Quote
[ 7]  Encode Gender
[ 8]  One-hot encode Country

The numbers presented in square brackets are the indices of the
corresponding pipeline stages, and they can be used to retrieve either the
specific pipeline stage objects composing the pipeline, e.g. with
`pipeline[5]`, or sub-pipelines composed of sub-sequences of
the pipeline, e.g. with `pipeline[2:6]`.

## Applying pipelines

The pipeline can now be applied to an input dataframe using the `apply`
method. We will also provide the `verbose` keyword with `True` to have a 
informative prints or the progress of dataframe processing, stage by stage:

In [6]:
res = pipeline.apply(df, verbose=True)

- Drop columns Name
- Drop rows by qualifier <RowQualifier: Qualify rows with df[Savings] >
  100>
2 rows dropped.
- Bin Savings by [1].


Savings: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 143.72it/s]

- Scale columns Columns of dtypes <class 'numpy.number'>
- Tokenize Quote
- Stemming tokens in Quote...
- Remove stopwords from Quote
- Encode Gender



100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 242.12it/s]

- One-hot encode Country



Country: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 212.84it/s]


We will thus get the dataframe below. We can see all numerical columns were scaled, the `Country` column was one-hot-encoded, `Savings` also got a binned version and the textual `Quote` column underwent some word-level manipulations:

In [7]:
res

,Age,Gender,Smoking,Savings,Savings_bin,Quote,Country_Germany,Country_Greece,Country_USA
0,-0.917257,1,True,-0.718473,<1,"[live, life, fullest]",0,0,1
1,-0.917257,0,True,-0.625375,<1,"[pen, mightier, sword]",0,0,1
2,-0.806074,1,False,0.184172,1≤,"[one, one]",0,1,0
3,0.250161,1,True,-0.301556,1≤,"[everi, life, precious]",0,0,0
4,1.806718,0,True,2.127084,1≤,"[get, porch]",0,1,0
5,0.583709,1,False,-0.665852,<1,"[boy, love, dog, cat]",1,0,0


## Fit and transform

Pipelines are also callable objects themselves, so calling `pipeline(df)` is 
equivalent to calling `pipeline.apply(df)`.

Additionally, pipelines inherently have a fit state. If none of the stages
composing them is fittable in nature this doesn't make a lot of a difference,
but many stage have a `fit_transform` vs `transform` logic, like encoders,
scalers and so forth.

>   *The `apply` pipeline method uses either `fit_transform` and `transform`
    in an intelligent and sensible way: If the pipeline is not fitted, calling
    it is equivalent to calling `fit_transform`, while if it is fitted, the
    call is practically a `transform` call.*

Let's say we want to utilize pdpipe's powerful slicing syntax to apply only
*some* of the pipeline stages to the raw dataframe. We will now use the 
`fit_transform` method of the pipeline itself to force all encompassed pipeline
stages to fit-transform themselves.

Here, we will use `pipeline[2:5]` to apply the binning, scaling and tokenization stages only:

In [8]:
pipeline[2:4].fit_transform(df)

,Age,Name,Gender,Smoking,Savings,Savings_bin,Country,Quote
0,-1.135052,Jo,M,True,-0.609615,<1,USA,Living life to its fullest
1,-1.135052,Dana,F,True,-0.604820,<1,USA,the pen is mightier then the sword
2,-1.049790,Bo,M,False,-0.563121,1≤,Greece,all for one and one for all
3,-0.239800,Derek,M,True,-0.588140,1≤,Denmark,every life is precious
4,0.953870,Regina,F,True,-0.463043,1≤,Greece,all of you get off my porch
5,0.015987,Jim,M,False,-0.606905,<1,Germany,boy do I love dogs and cats
6,1.294918,Richy,M,False,1.478052,1≤,Finland,I gots the dollarz
7,1.294918,Wealthus,F,False,1.957592,1≤,Finland,me likey them moniez
